# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*')) #join the file path & roots with subdirectories using glob

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print('The CSV file contains',sum(1 for line in f),'rows.')

The CSV file contains 6821 rows.


## Part II. Complete the Apache Cassandra coding portion of your project.

Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the workspace directory. This file contains the following columns:
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine (127.0.0.1)
from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# To create a Keyspace 
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkifydb 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")

#### Set Keyspace

In [7]:
session.set_keyspace('sparkifydb')

#### Create queries to answer the following three questions of the data

1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

## Query 1:
**Get the artist, song title, and the song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4.**

The primary key(sessionid, itemInSession) is set to provide a unique combination. The sessionid is used as the partition key so that all the information for a session is stored in the same node, instead of distributed across different nodes. itemInSession is set as clustering column to sort data if required.

#### Table Creation

In [8]:
query = "CREATE TABLE IF NOT EXISTS artist_song_length"
query = query + "(sessionid int, itemInSession int, artist text, song text, length float, PRIMARY KEY(sessionid, itemInSession))"
session.execute(query)

#### Data Insertion

In [9]:
## extract data from the csv tile and load into the Cassandra table 
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_song_length (sessionid, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## Assign column element for each column in the INSERT statement
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Data Validation

In [11]:
## set SELECT with selection criteria
query = "select artist, song, length from artist_song_length WHERE sessionid = 338 and itemInSession = 4"
rows = session.execute(query)

## set column names for dataframe and initialize an empty 'data' list
columns=('artist_name', 'song_title', 'song_length') 
data=[] 

## insert the list with query results
for row in rows:
    values = [row.artist, row.song, row.length] 
    data.append(dict(zip(columns, values)))
    
## create dataframe and print
df = pd.DataFrame(data,columns=columns) 
df

,artist_name,song_title,song_length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


## Query 2: 
**Get the name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182.**

The composite key(userId, sessionId) uniquely identifies a songplay for the above query; use itemInSession and sessionid as clustering columns for filtering and sorting.

#### Table Creation

In [12]:
query = "CREATE TABLE IF NOT EXISTS artist_song_user"
query = query + "(userid int, sessionid int, itemInSession int, artist text, song text, firstname text, lastname text, PRIMARY KEY((userid, sessionid), itemInSession))"
session.execute(query)

#### Data Insertion

In [13]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_song_user (userid, sessionid, itemInSession, artist, song, firstname, lastname)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Assign column element for each column in the INSERT statement
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### Data Validation

In [14]:
##where clause uses both fields in primary key and order by clause uses field in clustering key
query = "select artist, song, firstname, lastname from artist_song_user WHERE userId = 10 AND sessionId = 182 ORDER BY itemInSession ASC"
rows = session.execute(query)
    
## set column names for dataframe and initialize an empty 'data' list
columns=('artist_name', 'song_title', 'first_name', 'last_name') 
data=[] 

## insert the list with query results
for row in rows:
    values = [row.artist, row.song, row.firstname, row.lastname]
    data.append(dict(zip(columns, values)))
    
## create dataframe and print
df = pd.DataFrame(data,columns=columns) 
df

,artist_name,song_title,first_name,last_name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


## Query 3: 
**Get every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'.**

The primary key is (song, userId) to return a unique combination. Set the partition key to song to keep all users who listen to the same song on a single node. Use userId as a clustering column for filtering if needed.

#### Table Creation

In [15]:
query = "CREATE TABLE IF NOT EXISTS user_song "
query = query + "(song text, userId int, firstname text, lastname text, PRIMARY KEY(song, userId))"
session.execute(query)          

#### Data Insertion

In [16]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_song (song, userid, firstname, lastname)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4])) #assign column elements to Insert

#### Data Validation

In [17]:
query = "SELECT firstname, lastname from user_song WHERE song = 'All Hands Against His Own'"
rows = session.execute(query)
    
## set column names for dataframe and initialize an empty 'data' list
columns=('first_name', 'last_name') 
data=[] 

## insert the list with query results
for row in rows:
    values = [row.firstname, row.lastname]
    data.append(dict(zip(columns, values)))
    
## create dataframe and print
df = pd.DataFrame(data,columns=columns) 
df

,first_name,last_name
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [18]:
query = "DROP TABLE sparkifydb.artist_song_length"
session.execute(query)
    
query = "DROP TABLE sparkifydb.artist_song_user"
session.execute(query)
    
query = "DROP TABLE sparkifydb.user_song"
session.execute(query)

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()